## Lab 1. Forecasting Agent

### Introduction

In this notebook we show you how to create your first agent on Amazon Bedrock.

[Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/) streamline workflows and automate repetitive tasks.

This agent will be a forecasting agent, where customers can ask the agent to return information about their current energy consumption and forecast of it.

The following represents the piece of architecture that will be built on this module.

![Architecture](../img/lab1-architecture.png)

### Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure to have version superior of **1.35.45**

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install Dependencies from local package
!pip install ../boto3/botocore-1.35.55-py3-none-any.whl \
    ../boto3/boto3-1.35.55-py3-none-any.whl \
    ../boto3/awscli-1.35.21-py3-none-any.whl --force-reinstall --no-cache

In [ ]:
# Only install if running locally
!pip install pickleshare --force-reinstall --no-cache

Restart kernel for packages to take effect

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

Check if your boto3 version is superior than **1.35.45**

In [ ]:
!pip freeze | grep boto3

### Creating Agent

On this section we're going to declare global variables that will be act as helpers during entire notebook and you will start to create your first agent.

In [2]:
import boto3
import os
import sys

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

agent_foundation_model = ['anthropic.claude-3-5-sonnet-20240620-v1:0']
#agent_foundation_model = ['anthropic.claude-3-5-haiku-20241022-v1:0']

In [3]:
forecast_agent_name = f"forecast-agent-{account_id}"

forecast_lambda_name = f"fn-forecast-agent-{account_id}"

forecast_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{forecast_agent_name}'

On following section, we're adding `agents.py` on Python path, so the file can be recognized and invoked.

In [4]:
# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

parent_dir = os.path.dirname(current_dir)
#print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

Now, you're going to import from helper file `agents.py` AgentsForAmazonBedrock helper class.

This class is a helper totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by this class.

Following are methods that you're going to invoke on this lab:

- create_agent: Create a new agent and respective IAM roles
- add_action_group_with_lambda: Create a lambda function and add it as an action group for a previous created agent
- create_agent_alias: Create an alias for this agent
- invoke: Execute agent

In [5]:
from agent import AgentsForAmazonBedrock

agents = AgentsForAmazonBedrock()

Create the forecast agent

In [6]:
forecast_agent_id = agents.create_agent(
    forecast_agent_name, 
    """
        You are a energy usage forecast bot. 
        You can retrieve energy consumption forecast for a specific user
    """, 
    """
        You are a energy usage forecast bot.
        You can retrieve energy consumption forecast for a specific user. 
        Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        Never ask the user for information that you already can retrieve yourself through available actions. 

    """,
    agent_foundation_model
)

forecast_agent_id

'SZXNA1ZDHD'

### Creating Lambda

On this block, we're going to generate Lambda function Code:

In [8]:
%%writefile forecast.py
import json

def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def forecast_consumption(customer_id):
    # TODO: Implement real business logic
    return [
        {"day": "2024/06/01",
         "sumPowerReading": "120.0"
        },
        {"day": "2024/07/01",
         "sumPowerReading": "130.0"
        },
        {"day": "2024/08/01",
         "sumPowerReading": "140.0"
        },
        {"day": "2024/09/01",
         "sumPowerReading": "150.0"
        },
        {"day": "2024/10/01",
         "sumPowerReading": "200.0"
        },
        {"day": "2024/11/01",
         "sumPowerReading": "190.0"
        },
        {"day": "2024/12/01",
         "sumPowerReading": "205.0"
        },
        {"day": "2025/01/01",
         "sumPowerReading": "210.0"
        }
    ]

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    
    if function == 'forecast_consumption':
        customer_id = get_named_parameter(event, "customer_id")
        result = forecast_consumption(customer_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Writing forecast.py


Now it's time to add this Lambda function as an action group for this agent and prepare it.

In [9]:
resp = agents.add_action_group_with_lambda(
            forecast_agent_name, 
            forecast_lambda_name, 
            "forecast.py", 
            [
                {
                    "name": "forecast_consumption",
                    "description": """Retrieves usage consumption and projection 
                                      for next 3 months""",
                    "parameters": {
                                    "customer_id": {
                                        "description": "The ID of the customer",
                                        "required": True,
                                        "type": "string"
                                    }
                                }
                }
            ], 
            "forecast_consumption_actions", 
            "Function to get usage forecast for a user "
            )

### Create alias

Create an alias that will be used further on multi-agent collaborator feature:

In [11]:
agents_boto3 = boto3.client('bedrock-agent',
                            region_name=region)

forecast_agent_alias_resp = agents_boto3.create_agent_alias(agentId=forecast_agent_id, 
                                                           agentAliasName='v1')

In [ ]:
# create alias
#forecast_agent_alias_arn = agents.create_agent_alias(forecast_agent_id, 'v1')

### Invoking Agent

Now, let's run some tests on agent to make sure it's working:

In [15]:
%%time
print(agents.invoke("can you give me my forecasted energy consumption in a bar chart? My id is 45", 
                    forecast_agent_id)
)

Certainly! Here's an ASCII bar chart representing your forecasted energy consumption for the next 8 months:

Forecasted Energy Consumption (kWh) for Customer ID: 45

210 |                                       ██
200 |                   ██           ██     ██
190 |                   ██     ██    ██     ██
180 |                   ██     ██    ██     ██
170 |                   ██     ██    ██     ██
160 |                   ██     ██    ██     ██
150 |               ██  ██     ██    ██     ██
140 |           ██  ██  ██     ██    ██     ██
130 |       ██  ██  ██  ██     ██    ██     ██
120 |   ██  ██  ██  ██  ██     ██    ██     ██
110 |   ██  ██  ██  ██  ██     ██    ██     ██
100 |   ██  ██  ██  ██  ██     ██    ██     ██
 90 |   ██  ██  ██  ██  ██     ██    ██     ██
 80 |   ██  ██  ██  ██  ██     ██    ██     ██
 70 |   ██  ██  ██  ██  ██     ██    ██     ██
 60 |   ██  ██  ██  ██  ██     ██    ██     ██
 50 |   ██  ██  ██  ██  ██     ██    ██     ██
 40 |   ██  ██  ██  ██  ██     ██  

Store environment variables to be used on next notebooks.

In [ ]:
forecast_agent_arn = agents.get_agent_arn_by_name(forecast_agent_name)
forecast_agent_alias_arn = forecast_agent_alias_resp['agentAlias']['agentAliasArn']
forecast_agent_alias_id = forecast_agent_alias_resp['agentAlias']['agentAliasId']

%store forecast_agent_arn
%store forecast_agent_alias_arn
%store forecast_agent_alias_id
%store forecast_lambda_name
%store forecast_agent_name
%store forecast_agent_id

In [14]:
forecast_agent_arn, forecast_agent_alias_arn, forecast_agent_alias_id

('arn:aws:bedrock:us-west-2:471112746845:agent/SZXNA1ZDHD',
 'arn:aws:bedrock:us-west-2:471112746845:agent-alias/SZXNA1ZDHD/4PIPHQ8AV4',
 '4PIPHQ8AV4')

### Clean Up

In [5]:
agents.delete_lambda(forecast_lambda_name)

In [ ]:
agents.delete_agent_alias(forecast_agent_alias_id, forecast_agent_id)

In [ ]:
agents.delete_agent(forecast_agent_name)